In [1]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [7]:

from static_grader import grader


# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [8]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `pw` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a pandas `DataFrame` by making use of the `pd.read_csv` function to parse the data into a `DataFrame`. You may want to check the pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [`pw` miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [9]:
import pandas as pd
import numpy as np
import gzip

In [10]:
# load the 2017 data
scripts_file = gzip.open('./dw-data/201701scripts_sample.csv.gz', 'r')
    
scripts = pd.read_csv(scripts_file)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [11]:

practices_file = gzip.open('./dw-data/practices.csv.gz', 'r')

col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv(practices_file, names = col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [12]:
chem = pd.read_csv(gzip.open('./dw-data/chem.csv.gz', 'r'))
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `pw` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `pw` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `pw` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: `[('act_cost', (total, mean, std, q25, median, q75)), ...]`

In [13]:
def summary(key):
    total = scripts[key].sum()
    avg = scripts[key].describe()[1]
    s = scripts[key].describe()[2]
    q25 = scripts[key].describe()[4]
    med = scripts[key].median()
    q75 = scripts[key].describe()[6]
    return (total, avg, s, q25, med, q75)

summary_stats = [('items', summary('items')), 
                 ('quantity', summary('quantity')), 
                 ('nic', summary('nic')), 
                 ('act_cost', summary('act_cost'))]

In [14]:
grader.score.dw__summary_statistics(summary_stats)

Your score: 1.000


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [15]:
most_common_item = [("", 0)]

In [16]:
items = scripts.groupby(['bnf_name'], sort=False)['items'].sum()
items_df = pd.DataFrame({'bnf_name':items.index, 'items':items.values})
a = items_df.loc[items_df['items'].idxmax()]
most_common_item = []
empty_tuple = ()
empty_tuple = a[0],a[1]
most_common_item.append(empty_tuple)
print(most_common_item)

[('Omeprazole_Cap E/C 20mg', 218583)]


In [17]:
grader.score.dw__most_common_item(most_common_item)

Your score: 1.000


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the `pw` miniproject).

In [18]:

# Sort scripts by bnf_name and print the first 5 records in scripts
sort_scripts = scripts.sort_values('bnf_name')
sort_scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
824938,H84015,200301006,365 Film 10cm x 12cm VP Adh Film Dress,2,6.40,5.95,40
569412,M91614,200301006,365 Non Adherent 10cm x 10cm Pfa Plas Fa,1,1.80,1.68,20
932168,H81033,200301006,365 Non Adherent 10cm x 10cm Pfa Plas Fa,1,9.00,8.34,100
876439,L81051,200301006,365 Non Adherent 10cm x 10cm Pfa Plas Fa,1,0.90,0.85,10
638246,F81001,200301006,365 Non Adherent 10cm x 20cm Pfa Plas Fa,1,4.25,3.95,25


In [19]:
unique_practices = (practices.sort_values('post_code').groupby('code').first().reset_index())
unique_practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,MCKENZIE HOUSE,17 KENDAL ROAD,HARTLEPOOL,CLEVELAND,TS25 1QU
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,ACKLAM,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [20]:

# Join/Merge scripts with uique practices and print the first 5 records in it
joined = pd.merge(scripts, unique_practices, how = 'inner', left_on = 'practice', right_on = 'code')
joined.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [21]:
#Computes sum of groups and display the first 5 records in it
groups = joined.groupby(['post_code', 'bnf_name'])
compute_sums = groups.sum()
compute_sums.head()

items     nic  act_cost  \
post_code bnf_name                                                            
B11 4BW   3m Health Care_Cavilon Durable Barrier C      7   48.56     45.03   
          3m Health Care_Cavilon No Sting Barrier       2   11.96     11.10   
          Abasaglar KwikPen_100u/ml 3ml Pf Pen          2   70.56     65.35   
          Abidec_Dps                                   63  283.05    262.83   
          Able Spacer + Sml/Med Mask                    1    7.16      6.64   

                                                    quantity  
post_code bnf_name                                            
B11 4BW   3m Health Care_Cavilon Durable Barrier C         7  
          3m Health Care_Cavilon No Sting Barrier          2  
          Abasaglar KwikPen_100u/ml 3ml Pf Pen            10  
          Abidec_Dps                                    2125  
          Able Spacer + Sml/Med Mask                       1

In [22]:

# Display largest result
a = compute_sums['items'].groupby(level=0, group_keys=False)
largest_result = a.nlargest(1)
largest_result.head()

post_code  bnf_name                          
B11 4BW    Salbutamol_Inha 100mcg (200 D) CFF     706
B12 9LP    Paracet_Tab 500mg                      425
B18 7AL    Salbutamol_Inha 100mcg (200 D) CFF     556
B21 9RY    Metformin HCl_Tab 500mg               1033
B23 6DJ    Lansoprazole_Cap 30mg (E/C Gran)       599
Name: items, dtype: int64

In [23]:
# Compute sum and Display the first 5 records in post code groups
post_code_groups = joined.groupby(['post_code'])['items'].sum()
post_code_groups.head()

post_code
B11 4BW    22731
B12 9LP    17073
B18 7AL    20508
B21 9RY    31027
B23 6DJ    28011
Name: items, dtype: int64

In [24]:
#Iterate over largest result
total_portion = 0
entire_list = []
final_list=[]

count=0
for i, j in largest_result.iteritems():
    if count<100:
        for a, b in post_code_groups.iteritems():
            if a == i[0]:
                total_portion = j/b
        empty_tuple = ()

        empty_tuple = i[0],i[1],round(total_portion,10)
        entire_list.append(empty_tuple)
    count+=1
final_list = entire_list

items_by_region = final_list

In [25]:
grader.score.dw__items_by_region(items_by_region)

Your score: 1.000


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [26]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [28]:
mask = chem['NAME'].str.contains('|'.join(opioids), case=False)
opioid_codes = chem[mask]['CHEM SUB']
scripts['opioids'] = scripts['bnf_code'].isin(opioid_codes).astype(int)

opioids_per_practice = scripts.groupby('practice')['opioids'].mean()

opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: opioids, dtype: float64

In [29]:
relative_opioids_per_practice = opioids_per_practice - scripts['opioids'].mean()
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: opioids, dtype: float64

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [30]:
standard_error_per_practice = scripts['opioids'].std() / np.sqrt(scripts['practice'].value_counts())
opioid_scores = relative_opioids_per_practice / standard_error_per_practice

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_code, practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [31]:
top_opioids = opioid_scores.sort_values(ascending=False)[:100]
top_opioids = pd.DataFrame(top_opioids.rename('z_score')).reset_index()
top_opioids.columns = ['practice', 'z_score']

In [32]:
unique_practices = practices.groupby('code')['name'].min()
unique_practices = pd.DataFrame(unique_practices)

In [33]:
results = top_opioids.merge(unique_practices, 
                 how='left', 
                 left_on='practice', 
                 right_index=True)

In [34]:
results = results.merge(pd.DataFrame(scripts.groupby('practice').size().rename('n_scripts')), 
                       how='left', 
                       left_on='practice', 
                       right_index=True)

In [35]:

results = results[['practice', 'name', 'z_score', 'n_scripts']]

In [36]:
anomalies = list(tuple(zip(results['name'], results['z_score'], results['n_scripts'])))

In [37]:
anomalies = [tuple(r) for r in results.values]

In [38]:
grader.score.dw__script_anomalies(anomalies)

Your score: 1.000


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the growth rate in prescription rate from June 2016 to January 2017 for each `bnf_name`. The growth is defined as

$$
\text{growth rate} = \frac{x_1 - x_0}{x_0},
$$
where $x_1$ is the current value and $x_0$ is the previous value.


We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative growth rate) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [39]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()
resp = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()
resp.fillna(0, inplace=True)
resp = resp[resp['item16'] >= 50]
resp.sort_values(by='growth', ascending = False, inplace=True)
script_growth = list(pd.concat([resp.head(50), resp.tail(50)]).itertuples(index=False, name=None))

In [40]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [41]:
grader.score.dw__script_growth(script_growth)

Your score: 1.000


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [42]:
p = 1/scripts.bnf_code.nunique()

rates = scripts['bnf_code'].value_counts() / len(scripts)

mask = rates < .1 * p

rare_codes = rates[mask].index.unique()

scripts['rare'] = scripts['bnf_code'].isin(rare_codes)

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [43]:
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/scripts.groupby('practice')['act_cost'].sum()).fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [44]:
relative_rare_cost_prop = (rare_cost_prop - scripts[scripts['rare']]['act_cost'].sum() /scripts['act_cost'].sum())

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [45]:
standard_error = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(practice_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [46]:
rare_scores = relative_rare_cost_prop/standard_error
unique_practices = (practices.sort_values('name').groupby('code').first().reset_index())[['code', 'name']].rename(columns={'code':'practice'})

unique_practices.set_index('practice', inplace=True)

rare_scores = pd.concat([unique_practices, rare_scores], axis =1).sort_values(by='act_cost', ascending=False).head(100)

In [47]:
rare_scripts = list(rare_scores.itertuples(name=None))

In [48]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [49]:
grader.score.dw__rare_scripts(rare_scripts)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*